In [ ]:
import pandas as pd
import nltk
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
import torch

# Introducción

Para realizar una comparación entre los modelos clásicos y un modelo actual de procesamiento de lenguaje natural (NLP) se ha escogido el modelo de BERT, ya que es un modelo muy potente con el que es interesante realizar dicha comparación.

BERT (Bidirectional Encoder Representations from Transformers) es un modelo de lenguaje basado en redes neuronales usado para procesamiento de lenguaje natural . Funciona mediante el pre-entrenamiento en un corpus masivo de texto, lo que le permite capturar el contexto y la semántica del lenguaje de manera profunda. Posteriormente, el modelo puede ser ajustado o "ajustado fino" para tareas específicas de NLP, como la clasificación de texto, la extracción de información o la traducción automática. BERT es capaz de comprender el lenguaje en diferentes aplicaciones, lo que lo convierte en un hito en el campo del NLP. Su arquitectura se basa en la red transformer, y su capacidad para codificar el texto y obtener representaciones numéricas precisas lo hace extremadamente versátil y efectivo en una amplia gama de tareas de procesamiento del lenguaje natural.

En este trabajo se ha usado BERT para clasificar las páginas se ha usado el modelo BERT-base-uncased y el nlptown/bert-base-multilingual-uncased-sentiment.

BERT-base-uncased es una versión de BERT que ha sido pre-entrenada en un corpus masivo de texto en minúsculas. Ofrece la capacidad de comprender el contexto y la semántica del lenguaje en inglés, lo que lo hace adecuado para una amplia gama de tareas de procesamiento del lenguaje natural (NLP). El modelo es "uncased", lo que significa que no distingue entre mayúsculas y minúsculas.

El modelo "nlptown/bert-base-multilingual-uncased-sentiment" es una versión del modelo BERT (Bidirectional Encoder Representations from Transformers) que ha sido pre-entrenada para comprender el sentimiento en texto en varios idiomas. Es una red neuronal profunda que puede analizar el sentimiento en frases y párrafos en diferentes idiomas, lo que lo hace útil para tareas de procesamiento de lenguaje natural en un contexto multilingüe. El modelo es "uncased", lo que significa que no distingue entre mayúsculas y minúsculas.


Se descarga una librería de stopwords para la tokenización de los datos

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jatop\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Se cargan los datos con los que va a entrenar el modelo

In [ ]:
train_data = pd.read_json('datosParsed3.json')
#display(train_data)
# Selecciona una fila, omite la siguiente, y así sucesivamente


# Tokenizado y preparación de los datos

Aquí se tokenizan los datos a través de un bertTokenizer para el modelo de ber que vayamos a usar, se filtran las stopwords que se han obtenido anteriormente y se separa entre etiquetas y datos.

Además separamos los datos en lo que sería train y test  con una proporción 80% train 20% test. Una vez hecho esto se crean los tensores para poder trabajar con el modelo de bert.

In [ ]:
# Inicializar el tokenizador BERT con el modelo de bert que deseamos utilizar
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') #"nlptown/bert-base-multilingual-uncased-sentiment"

stop_words = set(nltk.corpus.stopwords.words('english'))
train_data['text'] = train_data['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

# Extraer textos y etiquetas
train_texts = train_data['text'].tolist()
train_labels = train_data['label'].tolist()

#Separar los datos entre train y test para el entrenamiento del modelo
X_train, X_test, y_train, y_test = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

# Tokenizar y convertir a tensores
train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(X_test, truncation=True, padding=True, return_tensors='pt')

# Mostrar información sobre las codificaciones
print(train_encodings.keys())
print(train_encodings['input_ids'].shape)
print(train_encodings['attention_mask'].shape)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([5298, 512])
torch.Size([5298, 512])


Para poder manejar los datos de forma cómoda se crea un dataloader, de esta forma tendremos los datos mejor repartidos en lotes y será mas cómodo para nuestro modelo trabajar con los mismos.

In [ ]:
# Crear un conjunto de datos personalizado
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        self.label_mapping = {'course': 0, 'department': 1, 'faculty': 2, 'other': 3, 'project': 4, 'staff': 5, 'student': 6}

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        # Convierte la etiqueta a tipo numérico usando el mapeo
        item['labels'] = torch.tensor(self.label_mapping[self.labels[idx]])

        return item



Creamos los conjuntos de datos y los dataloaders con la función definida arriba y establecemos el batch size

In [ ]:
# Crear conjuntos de datos y DataLoader
train_dataset = CustomDataset(train_encodings, y_train)
test_dataset = CustomDataset(test_encodings, y_test)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Entrenamiento del modelo

Primero se debe cargar el modelo de bert que deaseamos usar indicándole cuáles van a ser las etiquetas por las que va a tener que clasificar los datos.

Para que el modelo funcione de la forma más óptima posible se debería cargar en un dispositivo de GPU, pero en caso de no ser posible se cargara en la CPU.

Se define un optimizador para el modelo con los parámetros de la tasa de aprendizaje y la estabilidad numérica deseados.

In [ ]:
# Inicializar el modelo BERT elegido, bastoacon cambiar el parametro que indica el nombre del modelo
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=len(train_data['label'].unique()),
                                                      ignore_mismatched_sizes=True) #"nlptown/bert-base-multilingual-uncased-sentiment"

# Configurar el dispositivo a GPU si está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Inicializar el optimizador
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\jatop\anaconda3\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Se inicia el entrenamiento del modelo, para ello se declara el numero de épocas de entrenamiento.

In [ ]:
# Establecer el modelo en modo de entrenamiento
model.train()

# Definir la función de pérdida (criterio)
criterion = torch.nn.CrossEntropyLoss()

# Número de épocas (ajusta según sea necesario)
num_epochs = 3

# Bucle de entrenamiento
for epoch in range(num_epochs):
    for batch in train_loader:
        # Transferir datos al dispositivo (GPU si está disponible)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Realizar la propagación hacia adelante
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Realizar la retropropagación y la actualización de parámetros
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

C:\Users\jatop\AppData\Local\Temp\ipykernel_13928\1478713788.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


# Evaluación del modelo

Una vez que el modelo ha sido entrenado vemos qué resultados hemos obtenido con el conjunto de test, de forma que tendremos una aproximación de cómo de bien o mal ha funcionado nuestro modelo. Para ello se muestra un informe de clasificación con los datos obtenidos después de la evaluación

In [ ]:
# Cambiar el modelo a modo de evaluación
model.eval()

# Listas para almacenar las predicciones y etiquetas reales
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in train_loader:
        # Transferir datos al dispositivo (GPU si está disponible)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Realizar la propagación hacia adelante sin realizar la retropropagación
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Obtener las predicciones y las etiquetas reales
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        labels = labels.cpu().numpy()

        # Almacenar las predicciones y las etiquetas reales
        all_preds.extend(preds)
        all_labels.extend(labels)

# Calcular y mostrar el informe de clasificación
print(classification_report(all_labels, all_preds))

C:\Users\jatop\AppData\Local\Temp\ipykernel_13928\1478713788.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


              precision    recall  f1-score   support

           0       0.95      0.97      0.96       577
           1       0.98      0.87      0.92       119
           2       0.93      0.98      0.96       715
           3       0.98      0.96      0.97      2421
           4       0.96      0.94      0.95       309
           5       0.91      0.82      0.86        87
           6       0.95      0.96      0.95      1070

    accuracy                           0.96      5298
   macro avg       0.95      0.93      0.94      5298
weighted avg       0.96      0.96      0.96      5298



# Clasificación

Preparamos el conjunto de datos que queremos clasificar, de forma similar a como hicimos con los datos de entrenamiento y test.

In [ ]:
test_data = pd.read_json('dataTestParsed3.json')

# Preprocesar el conjunto de datos de prueba de la misma manera que el conjunto de entrenamiento
test_data['text'] = test_data['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

# Extraer textos y etiquetas (si están disponibles en el conjunto de prueba)
test_texts = test_data['text']

# Tokenizar y convertir a tensores
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, return_tensors='pt').to(device)

El modelo, entrenado anteriormente, realiza la clasificación del nuevo conjunto de datos.

In [ ]:
# Cambiar el modelo a modo de evaluación
model.eval()

# Listas para almacenar las predicciones en el conjunto de prueba sin etiquetas
test_preds_without_labels = []

with torch.no_grad():
    for i in range(len(test_encodings['input_ids'])):
        # Transferir datos al dispositivo (GPU si está disponible)
        input_ids = test_encodings['input_ids'][i].unsqueeze(0).to(device)
        attention_mask = test_encodings['attention_mask'][i].unsqueeze(0).to(device)

        # Realizar la propagación hacia adelante sin realizar la retropropagación
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Obtener las predicciones
        preds = torch.argmax(logits, dim=1).cpu().numpy()

        # Almacenar las predicciones
        test_preds_without_labels.extend(preds)

# Obtener los nombres de los archivos (id) del conjunto de prueba
test_ids = test_data['id']

# Entrega

Creamos el documento CSV con el que se compararan la clasificación del modelo con la realidad.

In [ ]:
# Mapear las predicciones a las etiquetas reales usando el diccionario inverso de la asignación de etiquetas
label_mapping = {train_dataset.label_mapping[label]: label for label in train_dataset.label_mapping}

# Crear un DataFrame con las predicciones
predictions_df = pd.DataFrame({
    'id': test_ids,
    'label': [label_mapping[pred] for pred in test_preds_without_labels]
})

# Mostrar el DataFrame con las predicciones
print(predictions_df)

nombre_archivo = 'ENXEBRE-bert-uncased-bs-1.csv'
predictions_df.to_csv(nombre_archivo, index=False)

           id    label
0     aaclkul  student
1     aagelci    other
2     aangjmn    other
3      aawnpc    other
4     abdjgiz  student
...       ...      ...
1654    zxmmn    other
1655   zxwkru    other
1656  zybimtt    other
1657  zypnixf  project
1658   zzszho  student

[1659 rows x 2 columns]


Guardamos el modelo en un pickle.

In [ ]:
from transformers import BertModel
# Guardar el modelo
model.save_pretrained('C:\\Users\\jatop\\master\\Noestructurados')
